In [ ]:
# import libraries
import math
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# import tensorflow libraries
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, Dense, Flatten

In [ ]:
# import bayesian optimisation libraries
import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.plots import plot_histogram, plot_objective_2D
from skopt.utils import use_named_args
import h5py

In [2]:
# set hyperparameter tuning upper and lower bounds
dim_learning_rate = Real(low=1e-5, high=1e-2, prior='log-uniform',
                         name='learning_rate')
dim_num_dense_layers1 = Integer(low=1, high=5, name='num_dense_layers1')
dim_num_dense_layers2 = Integer(low=1, high=5, name='num_dense_layers2')
dim_num_dense_layers3 = Integer(low=1, high=7, name='num_dense_layers3')
dim_num_dense_nodes1 = Integer(low=5, high=256, name='num_dense_nodes1')
dim_num_dense_nodes2 = Integer(low=5, high=256, name='num_dense_nodes2')
dim_num_dense_nodes3 = Integer(low=5, high=256, name='num_dense_nodes3')
dim_num_batch = Integer(low=16, high=512, name='num_batch')

In [3]:
# concatenate all hyperparameters that require tuning
dimensions = [dim_learning_rate,
              dim_num_dense_layers1, dim_num_dense_layers2, dim_num_dense_layers3,
              dim_num_dense_nodes1, dim_num_dense_nodes2, dim_num_dense_nodes3,
              dim_num_batch]


In [4]:
# use manual hyperparameter selection for default parameters
default_parameters = [1e-5, 3,3,5, 16,250,250, 64]


In [5]:
# define log directory to view models
def log_dir_name(learning_rate,
                 num_dense_layers1, num_dense_layers2, num_dense_layers3,
                 num_dense_nodes1, num_dense_nodes2, num_dense_nodes3,
                 num_batch):

    # The dir-name for the TensorBoard log-dir.
    s = "./Bayes_SABR_param_sep_logs/lr_{0:.0e}_layer1_{1}_layer2_{2}_layer3_{3}_nodes1_{4}_nodes2_{5}_nodes3_{6}_batch_{7}/"

    # Insert all the hyper-parameters in the dir-name.
    log_dir = s.format(learning_rate,
                       num_dense_layers1, num_dense_layers2, num_dense_layers3,
                       num_dense_nodes1, num_dense_nodes2, num_dense_nodes3,
                       num_batch)

    return log_dir


In [6]:
# data preparation
from sklearn.preprocessing import StandardScaler
import pandas as pd

data = pd.read_csv("Data_SABR_vol_mult_rst.txt", header=None)
targets = data.iloc[:, 0:3]
data = data.iloc[:, 3:]
scaler = StandardScaler()
data = scaler.fit_transform(data)
train_inputs, test_inputs, train_targets, test_targets = train_test_split(
    data, targets, test_size=0.3,shuffle=False)
train_inputs = tf.convert_to_tensor(train_inputs, dtype='float64')
train_targets = tf.convert_to_tensor(train_targets.values, dtype='float64')
test_inputs = tf.convert_to_tensor(test_inputs, dtype='float64')
test_targets = tf.convert_to_tensor(test_targets.values, dtype='float64')
input_size = train_inputs.shape[1]
output_size = 3

In [7]:
# define early stopping mechanism
LRScheduler = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=10,
                                verbose=0, min_delta=0.000001, cooldown=10, min_lr=0.00001)


In [8]:
# define customized loss function
def my_loss_fn(y_true, y_pred):
    squared_difference = tf.square(y_true - y_pred)
    squared_difference = squared_difference * [0.25, 0.25, 0.5]
    return tf.reduce_mean(squared_difference, axis=-1)  # Note the `axis=-1`


In [9]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Concatenate

In [10]:
def create_model(learning_rate, num_dense_layers1, num_dense_layers2, num_dense_layers3,
                    num_dense_nodes1, num_dense_nodes2, num_dense_nodes3,num_batch):
   
     
        input = Input(shape=(36,))
        
        x_1 = Dense(num_dense_nodes1, activation="relu")(input)
        for i in range(1,num_dense_layers1-1):

            x_1 =Dense(num_dense_nodes1,
                            activation="relu")(x_1)
        x_2 = Dense(num_dense_nodes2, activation="relu")(input)
        for i in range(num_dense_layers2-1):

            x_2 = Dense(num_dense_nodes2,
                        activation="relu")(x_2)
        x_3 = Dense(num_dense_nodes3,activation="relu")(input)
        for i in range(num_dense_layers3-1):
            x_3 = Dense(num_dense_nodes3,
                        activation="relu")(x_3)


        x = Concatenate()([x_1, x_2, x_3])
        x = Dense(8, activation="relu")(x)
        output = Dense(3)(x)
        model = Model(inputs=[input], outputs=[output])
        optimizer = Adam(lr=learning_rate)

    # Compile the model so it can be trained.
        model.compile(optimizer="adam", loss=my_loss_fn,
                metrics=["mean_absolute_error"])

        return model




In [11]:
# initialise loss and best model directory
best_loss = 99999999
best_model_dir = ""


In [ ]:
# bayesian optimisation objective function
@use_named_args(dimensions=dimensions)
def fitness(learning_rate, num_dense_layers1, num_dense_layers2, num_dense_layers3, num_dense_nodes1, num_dense_nodes2, num_dense_nodes3, num_batch):

  # Print the hyper-parameters.
  print('learning rate: {0:.1e}'.format(learning_rate))
  print('num_dense_layers1:', num_dense_layers1)
  print('num_dense_layers2:', num_dense_layers2)
  print('num_dense_layers3:', num_dense_layers3)
  print('num_dense_nodes1:', num_dense_nodes1)
  print('num_dense_nodes2:', num_dense_nodes2)
  print('num_dense_nodes3:', num_dense_nodes3)
  print('num_batch:', num_batch)

  # Create the neural network with these hyper-parameters.
  model = create_model(learning_rate=learning_rate,
                        num_dense_layers1=num_dense_layers1, num_dense_layers2=num_dense_layers2, num_dense_layers3=num_dense_layers3,
                        num_dense_nodes1=num_dense_nodes1, num_dense_nodes2=num_dense_nodes2, num_dense_nodes3=num_dense_nodes3,
                        num_batch=num_batch)
    
  # Dir-name for the TensorBoard log-files.
  log_dir=log_dir_name(learning_rate, num_dense_layers1, num_dense_layers2, num_dense_layers3,
                          num_dense_nodes1, num_dense_nodes2, num_dense_nodes3, num_batch)



  callback_log = TensorBoard(
      log_dir=log_dir)

  # Use Keras to train the model.
  history = model.fit(train_inputs,  
                      train_targets,  
                      batch_size=num_batch,  
                      epochs=250,
                      validation_split=0.1,
                      callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=40, verbose=False, min_delta=0.000001),
                                  LRScheduler,
                                  callback_log],
                    
                      verbose=True)

  # Get the loss on the validation-set
  loss = history.history['val_loss'][-1]

  # Print the loss
  print("Loss: {0}".format(loss))

  # Save the model if it improves on the best-found performance.
  global best_loss
  global best_model_dir
  # If the lpss of the saved model is improved.
  if loss < best_loss:
      # Save the new model
      if best_model_dir == '':
        pass
      else:
        os.remove(best_model_dir)

      best_model_dir = os.path.join(log_dir, 'best_model.h5')
      model.save(best_model_dir)
      # Update the loss
      best_loss = loss

  print(best_model_dir)
  # Delete the Keras model with these hyper-parameters from memory.
  del model

  K.clear_session()

  return loss


In [ ]:
# test function is working
fitness(x=default_parameters)


In [ ]:
# Begin bayesian optimization
search_result = gp_minimize(func=fitness,
                            dimensions=dimensions,
                            n_calls=40,
                            x0=default_parameters)